In [1]:
import folium
import sys
from datetime import datetime
import utils
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display
from geopy import distance
from multiprocessing import Pool
from research_2 import *

In [2]:
# data = utils.read_dmp_data("data/0322-0409_batchE_top.csv")
# data_home_and_work = utils.read_home_work_data("data/0322-0409_batchE_HW.csv")
fix_map = False
epsilon = sys.float_info.epsilon
# home_work_node = data_home_and_work.query(f'id == "{uuid}"').to_dict(orient='records')[0]
# home_work_node

In [3]:
csv_file_path = "../data/0322-0409_batchA.csv"
date_str = '2023-03-31'
uuid = "510016232995249"
list_threshold_numbers = [1.0, -0.5, -0.8] 

#NOTE: some IDs have NO point left after -0.5 and -0.8 thresholds
# "510010342736250", "510012433336951" 

selet_csv_by_date(csv_file_path, date_str)
csv_file_path_Mar31 = '../data/' + date_str + 'new_file.csv'
data = utils.read_dmp_data(csv_file_path_Mar31)
footprint = data[uuid]
for th_num in list_threshold_numbers:
    render_cosined_map(csv_file_path_Mar31, uuid, th_num)

510016232995249
POINT COUNT 91
510016232995249
POINT COUNT 32
510016232995249
POINT COUNT 25


### BORDER

In [4]:
import math

# Cosine value
cos_value = -0.8

# Calculate the angle in radians
angle_rad = math.acos(cos_value)

# Convert radians to degrees
angle_deg = math.degrees(angle_rad)

print(f"Angle: {angle_deg} degrees")


Angle: 143.13010235415598 degrees


In [5]:
# pipeline("510018345732277")
# pipeline("510018242552475")